In [18]:
# =============================================================================
# CELL 1: Setup and Imports
# =============================================================================

# Add Python_files directory to path
import sys
import os
sys.path.append('../Python_files')
#sys.path.append(os.path.join('..', 'Python_files'))

# Import required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from data_loader import load_and_validate
from data_processor import process_all_data
from analysis_engine import run_complete_analysis
from visualization import create_all_visualizations, save_all_plots, display_figures_in_notebook

print("✓ All modules imported successfully")
print("✓ Ready to start analysis!")
print(f"✓ Working directory: {os.getcwd()}")

✓ All modules imported successfully
✓ Ready to start analysis!
✓ Working directory: c:\Dissertation_Term3\MScProject\Notebooks


In [19]:
# =============================================================================
# CELL 2: Data Loading and Validation
# =============================================================================

# Load and validate all data
data, is_valid = load_and_validate()

if not is_valid:
    raise ValueError("Data validation failed. Please check your data files in the Data folder.")

print("\n" + "="*70)
print("DATA LOADING SUMMARY")
print("="*70)
for name, df in data.items():
    memory_usage = df.memory_usage(deep=True).sum() / 1024**2
    print(f"{name:20}: {df.shape[0]:,} rows × {df.shape[1]:2d} columns ({memory_usage:.1f} MB)")

total_memory = sum(df.memory_usage(deep=True).sum() for df in data.values()) / 1024**2
print(f"\nTotal memory usage: {total_memory:.1f} MB")

LOADING STUDENT ASSESSMENT DATA
Loading data from: ..\Data
✓ Loaded assessments.csv: (206, 6)
✓ Loaded courses.csv: (22, 3)
✓ Loaded studentAssessment.csv: (173912, 5)
✓ Loaded studentInfo.csv: (32593, 12)
✓ Loaded studentRegistration.csv: (32593, 5)
✓ Loaded studentVle.csv: (10655280, 6)
✓ Loaded vle.csv: (6364, 6)

Validating data structure...
✓ assessments has all required columns
✓ student_assessment has all required columns
✓ student_registration has all required columns
✓ student_vle has all required columns
✓ student_info has all required columns

✓ All data validation checks passed

DATA SUMMARY

ASSESSMENTS:
  Shape: (206, 6)
  Columns: ['code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'weight']
  Missing values: 11

COURSES:
  Shape: (22, 3)
  Columns: ['code_module', 'code_presentation', 'module_presentation_length']
  Missing values: None

STUDENT ASSESSMENT:
  Shape: (173912, 5)
  Columns: ['id_assessment', 'id_student', 'date_submitted', 'is

In [20]:
# =============================================================================
# CELL 3: Data Processing Pipeline
# =============================================================================

# Process all data through the complete pipeline
processed_data = process_all_data(data)

# Display processing results
print("\n" + "="*70)
print("PROCESSED DATASET OVERVIEW")
print("="*70)
print(f"Shape: {processed_data.shape}")
print(f"Memory usage: {processed_data.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"\nColumns ({len(processed_data.columns)}):")
for i, col in enumerate(processed_data.columns, 1):
    print(f"  {i:2d}. {col}")


STARTING DATA PROCESSING PIPELINE
Finding first assessments...
  Found 106 TMA assessments
✓ Found 22 first assessments across 22 courses
Filtering active students...
  Students before filtering: 32,593
✓ Active students after filtering: 27,725
⚠ Found 6 students with missing registration dates
Merging assessment scores...
  Students with scores: 24,155
  Students without scores: 3,570
✓ Merged assessment scores for 27,725 students
Calculating VLE engagement...
  Total VLE interactions before filtering: 10,567,484
  VLE interactions before first assessment: 2,587,468
  Students with no VLE activity: 981
✓ Calculated VLE engagement for 27,725 students
Creating engagement features...
  Students with excellent scores (>= 70): 16,687
  Students active in VLE: 9,870
  Overall engaged students: 19,176
✓ Created engagement features
Merging student demographic information...
✓ Final merged dataset: (27725, 24)
  Final result distribution:
    Pass: 12,361 (44.6%)
    Fail: 7,044 (25.4%)
    W

In [21]:
# =============================================================================
# CELL 4: Adding course per term
# =============================================================================

# Create a new column 'courses_per_term' that counts the number of courses per student per term
processed_data['courses_per_term'] = (
    processed_data
    .groupby(['id_student', 'code_presentation'])['code_module']
    .transform('nunique')
)

# Filter for students taking more than one course in a term
multi_course = processed_data[processed_data['courses_per_term'] > 1]

# Show relevant columns for clarity
multi_course[['id_student', 'code_presentation', 'code_module', 'courses_per_term']]

,id_student,code_presentation,code_module,courses_per_term
1591,538232,2013B,BBB,2
1617,540530,2013B,BBB,2
7274,29820,2014B,CCC,2
7279,49119,2014B,CCC,2
7285,64296,2014B,CCC,2
...,...,...,...,...
25169,1658963,2014J,FFF,2
25272,2470326,2014J,FFF,2
25293,2559144,2014J,FFF,2
25298,2599883,2014J,FFF,2


In [22]:
# =============================================================================
# CELL 5: Display Processed Data Sample
# =============================================================================

# Show sample of the processed data
print("SAMPLE OF PROCESSED DATA:")
print("="*50)
display(processed_data.head(10))

print("\nKEY STATISTICS:")
print("="*30)
key_columns = ['score', 'total_click_vle', 'average_click_vle', 'excellent_Score', 'active_in_VLE']
display(processed_data[key_columns].describe())

SAMPLE OF PROCESSED DATA:


,code_module,code_presentation,id_student,date_registration,date_unregistration,id_assessment,date_first_assessment,date_submitted,is_banked,score,...,gender,region,highest_education,imd_band,age_band,studied_credits,disability,final_result,final_result_code,courses_per_term
0,AAA,2013J,11391,-159.0,NaN,1752,19,18.0,0.0,78.0,...,M,East Anglian Region,HE Qualification,90-100%,55<=,240,N,Pass,2,1
1,AAA,2013J,28400,-53.0,NaN,1752,19,22.0,0.0,70.0,...,F,Scotland,HE Qualification,20-30%,35-55,60,N,Pass,2,1
2,AAA,2013J,31604,-52.0,NaN,1752,19,17.0,0.0,72.0,...,F,South East Region,A Level or Equivalent,50-60%,35-55,60,N,Pass,2,1
3,AAA,2013J,32885,-176.0,NaN,1752,19,26.0,0.0,69.0,...,F,West Midlands Region,Lower Than A Level,50-60%,0-35,60,N,Pass,2,1
4,AAA,2013J,38053,-110.0,NaN,1752,19,19.0,0.0,79.0,...,M,Wales,A Level or Equivalent,80-90%,35-55,60,N,Pass,2,1
5,AAA,2013J,45462,-67.0,NaN,1752,19,20.0,0.0,70.0,...,M,Scotland,HE Qualification,30-40%,0-35,60,N,Pass,2,1
6,AAA,2013J,45642,-29.0,NaN,1752,19,18.0,0.0,72.0,...,F,North Western Region,A Level or Equivalent,90-100%,0-35,120,N,Pass,2,1
7,AAA,2013J,52130,-33.0,NaN,1752,19,19.0,0.0,72.0,...,F,East Anglian Region,A Level or Equivalent,70-80%,0-35,90,N,Pass,2,1
8,AAA,2013J,53025,-179.0,NaN,1752,19,9.0,0.0,71.0,...,M,North Region,Post Graduate Qualification,NaN,55<=,60,N,Pass,2,1
9,AAA,2013J,57506,-103.0,NaN,1752,19,18.0,0.0,68.0,...,M,South Region,Lower Than A Level,70-80%,35-55,60,N,Pass,2,1



KEY STATISTICS:


,score,total_click_vle,average_click_vle,excellent_Score,active_in_VLE
count,27725.000000,27725.000000,27725.000000,27725.000000,27725.000000
mean,64.635564,329.356213,329.353133,0.601876,0.355996
std,31.174086,388.209551,152.890566,0.489520,0.478823
min,0.000000,0.000000,106.300000,0.000000,0.000000
25%,57.000000,85.000000,162.400000,0.000000,0.000000
50%,75.000000,215.000000,337.400000,1.000000,0.000000
75%,86.000000,442.000000,482.100000,1.000000,1.000000
max,100.000000,7129.000000,559.600000,1.000000,1.000000


In [24]:
# =============================================================================
# CELL 6: Data Quality Check
# =============================================================================

# Comprehensive data quality check
print("DATA QUALITY ASSESSMENT:")
print("="*50)

# Missing values
missing_values = processed_data.isnull().sum()
if missing_values.sum() > 0:
    print("Missing values by column:")
    for col, missing in missing_values[missing_values > 0].items():
        percentage = missing / len(processed_data) * 100
        print(f"  {col}: {missing:,} ({percentage:.1f}%)")
else:
    print("✓ No missing values found")

# Basic statistics
print(f"\nDATASET OVERVIEW:")
print(f"  Total students: {len(processed_data):,}")
print(f"  Unique courses: {processed_data['code_module'].nunique()}")
print(f"  Unique presentations: {processed_data['code_presentation'].nunique()}")
print(f"  Score range: {processed_data['score'].min():.1f} - {processed_data['score'].max():.1f}")
print(f"  VLE clicks range: {processed_data['total_click_vle'].min()} - {processed_data['total_click_vle'].max():,}")

# Multi-course students per cohort (code_module + code_presentation)
print(f"\nMULTI-COURSE STUDENTS PER COHORT:")
# Create temporary cohort column for analysis only
temp_cohort = processed_data['code_module'] + '_' + processed_data['code_presentation']

multi_course_by_cohort = processed_data[processed_data['courses_per_term'] > 1].groupby(temp_cohort).size()
if len(multi_course_by_cohort) > 0:
    print("Students taking multiple courses:")
    for cohort, count in multi_course_by_cohort.items():
        total_in_cohort = len(processed_data[temp_cohort == cohort])
        percentage = count / total_in_cohort * 100
        print(f"  {cohort}: {count:,} students ({percentage:.1f}%)")
else:
    print("  No students taking multiple courses in any cohort")

# Final result distribution
print("\nFINAL RESULT DISTRIBUTION:")
result_dist = processed_data['final_result'].value_counts().sort_index()
for result, count in result_dist.items():
    percentage = count / len(processed_data) * 100
    print(f"  {result:12}: {count:,} ({percentage:.1f}%)")

DATA QUALITY ASSESSMENT:
Missing values by column:
  date_registration: 7 (0.0%)
  date_unregistration: 22,521 (81.2%)
  date_submitted: 3,538 (12.8%)
  imd_band: 1,029 (3.7%)

DATASET OVERVIEW:
  Total students: 27,725
  Unique courses: 7
  Unique presentations: 4
  Score range: 0.0 - 100.0
  VLE clicks range: 0 - 7,129

MULTI-COURSE STUDENTS PER COHORT:
Students taking multiple courses:
  BBB_2013B: 2 students (0.1%)
  CCC_2014B: 288 students (19.5%)
  CCC_2014J: 337 students (16.9%)
  DDD_2013B: 2 students (0.2%)
  DDD_2013J: 4 students (0.2%)
  DDD_2014B: 37 students (3.6%)
  DDD_2014J: 94 students (6.4%)
  EEE_2013J: 2 students (0.2%)
  EEE_2014B: 217 students (35.9%)
  EEE_2014J: 176 students (17.3%)
  FFF_2013J: 2 students (0.1%)
  FFF_2014B: 36 students (2.8%)
  FFF_2014J: 73 students (3.9%)

FINAL RESULT DISTRIBUTION:
  Distinction : 3,024 (10.9%)
  Fail        : 7,044 (25.4%)
  Pass        : 12,361 (44.6%)
  Withdrawn   : 5,296 (19.1%)


In [ ]:
# =============================================================================
# CELL 7: Number of Courses per Term (code_presentation)
# =============================================================================

print("COURSE ANALYSIS BY TERM (CODE_PRESENTATION):")
print("="*50)

# Count unique courses per presentation term
courses_per_term = processed_data.groupby('code_presentation')['code_module'].nunique().sort_values(ascending=False)

print(f"OVERVIEW:")
print(f"  Total presentation terms: {len(courses_per_term)}")
print(f"  Total unique course modules: {processed_data['code_module'].nunique()}")

print(f"\nCOURSES PER TERM BREAKDOWN:")
for term, course_count in courses_per_term.items():
    # Also show total student enrollments for context
    total_enrollments = len(processed_data[processed_data['code_presentation'] == term])
    print(f"  {term}: {course_count} courses ({total_enrollments:,} total enrollments)")

# Additional analysis: Which terms had the most/least course variety
print(f"\nTERM INSIGHTS:")
max_term = courses_per_term.index[0]
min_term = courses_per_term.index[-1]
print(f"  Most course variety: {max_term} ({courses_per_term.iloc[0]} courses)")
print(f"  Least course variety: {min_term} ({courses_per_term.iloc[-1]} courses)")

# Show which specific courses were offered in each term (optional detailed view)
print(f"\nDETAILED COURSE BREAKDOWN BY TERM:")
print("-" * 40)
for term in sorted(courses_per_term.index):
    courses_in_term = processed_data[processed_data['code_presentation'] == term]['code_module'].unique()
    course_list = ', '.join(sorted(courses_in_term))
    print(f"{term} ({len(courses_in_term)} courses): {course_list}")

COURSE ANALYSIS BY TERM (CODE_PRESENTATION):
OVERVIEW:
  Total presentation terms: 4
  Total unique course modules: 7

COURSES PER TERM BREAKDOWN:
  2014J: 7 courses (9,201 total enrollments)
  2013J: 6 courses (7,709 total enrollments)
  2014B: 6 courses (6,516 total enrollments)
  2013B: 3 courses (4,299 total enrollments)

TERM INSIGHTS:
  Most course variety: 2014J (7 courses)
  Least course variety: 2013B (3 courses)

DETAILED COURSE BREAKDOWN BY TERM:
----------------------------------------
2013B (3 courses): BBB, DDD, FFF
2013J (6 courses): AAA, BBB, DDD, EEE, FFF, GGG
2014B (6 courses): BBB, CCC, DDD, EEE, FFF, GGG
2014J (7 courses): AAA, BBB, CCC, DDD, EEE, FFF, GGG

SUMMARY STATISTICS:
  Min courses per term: 3
  Max courses per term: 7
  Median courses per term: 6.0
  Standard deviation: 1.73


In [ ]:
# =============================================================================
# CELL 8: Save Processed Data
# =============================================================================

# Save processed_data to Data/output folder
import os

# Create output directory and save file
output_dir = os.path.join('..', 'Data', 'output')
os.makedirs(output_dir, exist_ok=True)

if 'processed_data' in locals():
    # Save the CSV file
    filepath = os.path.join(output_dir, 'processed_data.csv')
    processed_data.to_csv(filepath, index=False)
    
    print(f"✅ Saved processed_data to: {filepath}")
    print(f"📊 Shape: {processed_data.shape}")
    print(f"💾 File size: {os.path.getsize(filepath) / 1024**2:.1f} MB")
else:
    print("❌ processed_data not found. Please run data processing first.")

✅ Saved processed_data to: ..\Data\output\processed_data.csv
📊 Shape: (27725, 25)
💾 File size: 3.4 MB
